### [Embedding Metadata for Improved Retrieval](https://haystack.deepset.ai/tutorials/39_embedding_metadata_for_improved_retrieval)

In [1]:
#! Solo es para haystack sepa que tutorial se esta ejecutando
from haystack.telemetry import tutorial_running
tutorial_running(39)

### 1. Indexing documents with metadata

In [2]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

embedder = SentenceTransformersDocumentEmbedder(meta_fields_to_embed=["url"])

In [3]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice


def create_indexing_pipeline(document_store, metadata_fields_to_embed=None):
    document_cleaner = DocumentCleaner()
    document_splitter = DocumentSplitter(split_by="sentence", split_length=2)
    document_embedder = SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large", meta_fields_to_embed=metadata_fields_to_embed
    )
    document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("cleaner", document_cleaner)
    indexing_pipeline.add_component("splitter", document_splitter)
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", document_writer)

    indexing_pipeline.connect("cleaner", "splitter")
    indexing_pipeline.connect("splitter", "embedder")
    indexing_pipeline.connect("embedder", "writer")

    return indexing_pipeline

El **indexing_pipeline**: que indexa únicamente el contenido de los documentos. Indexaremos estos documentos en document_store.

El **indexing_with_metadata_pipeline**: que indexa los campos meta junto con el contenido de los documentos. Indexaremos estos documentos en document_store_with_embedded_metadata.

In [4]:
import wikipedia
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

some_bands = """The Beatles,The Cure""".split(",")

raw_docs = []

for title in some_bands:
    page = wikipedia.page(title=title, auto_suggest=False)
    doc = Document(content=page.content, meta={"title": page.title, "url": page.url})
    raw_docs.append(doc)

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
document_store_with_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")

indexing_pipeline = create_indexing_pipeline(document_store=document_store)
indexing_with_metadata_pipeline = create_indexing_pipeline(
    document_store=document_store_with_embedded_metadata, metadata_fields_to_embed=["title"]
)

indexing_pipeline.run({"cleaner": {"documents": raw_docs}})
indexing_with_metadata_pipeline.run({"cleaner": {"documents": raw_docs}})

/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


Batches:   0%|          | 0/17 [00:00<?, ?it/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

{'writer': {'documents_written': 542}}

### 2. Comparación de la recuperación con y sin metadatos integrados

**Primero:** recuperar desde el document_store, donde no hemos incrustado metadatos.  
**Segundo:** recuperar desde el document_store_with_embedded_metadata, donde hemos incrustado metadatos.

In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(model="thenlper/gte-large"))
retrieval_pipeline.add_component(
    "retriever", InMemoryEmbeddingRetriever(document_store=document_store, scale_score=False, top_k=3)
)
retrieval_pipeline.add_component(
    "retriever_with_embeddings",
    InMemoryEmbeddingRetriever(document_store=document_store_with_embedded_metadata, scale_score=False, top_k=3),
)

retrieval_pipeline.connect("text_embedder", "retriever")
retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - retriever_with_embeddings: InMemoryEmbeddingRetriever
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - text_embedder.embedding -> retriever_with_embeddings.query_embedding (List[float])

Ejecutemos la secuencia de comandos y comparemos los resultados de retrievery retirever_with_embeddings. A continuación, verá 3 documentos devueltos por cada recuperador, clasificados por relevancia.

In [6]:
result = retrieval_pipeline.run({"text_embedder": {"text": "Have the Beatles ever been to Bangor?"}})

print("Retriever Results:\n")
for doc in result["retriever"]["documents"]:
    print(doc)

print("Retriever with Embeddings Results:\n")
for doc in result["retriever_with_embeddings"]["documents"]:
    print(doc)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retriever Results:

Document(id=fe653f080a916754574e398d1cb1ef9a68a8203bae352b9945660b251d328700, content: ' The band flew to Florida, where they appeared on The Ed Sullivan Show a second time, again before 7...', meta: {'title': 'The Beatles', 'url': 'https://en.wikipedia.org/wiki/The_Beatles', 'source_id': 'db40eba373f4495f1d6c8048713e23fab35097f05a8e2a5b3782b0c1a82125cb', 'page_number': 1, 'split_id': 81, 'split_idx_start': 20230}, score: 0.8637016087645685)
Document(id=3935e7714360bc7b80d7ee463d7930fac9f9478249b54df56a408663ceac7aea, content: '
During the 1964 US tour, the group were confronted with racial segregation in the country at the ti...', meta: {'title': 'The Beatles', 'url': 'https://en.wikipedia.org/wiki/The_Beatles', 'source_id': 'db40eba373f4495f1d6c8048713e23fab35097f05a8e2a5b3782b0c1a82125cb', 'page_number': 1, 'split_id': 93, 'split_idx_start': 24734}, score: 0.85589534433623)
Document(id=3999a9640d430ca222fc9b324a891a48d4e9de202b44be080112f3f3355be9e8, content: '
I